# Demo for CLIFuzz

## What is CLIFuzz?

`CLIFuzz` is a tool to fuzz command-line interface (CLI) utilities using their configuration options and arguments. It first automatically extracts the configuration options and arguments from CLI utilities which use `getopt`, `getopt_long` or `getopt_long_only` to parse its CLI invocations. It then constructs a JSON grammar representing its invocation language which is then used to grammar fuzz the utility.

In [1]:
from IPython.display import IFrame

IFrame(src="https://www.youtube.com/embed/x8hAYCvVUfE", title="YouTube video player", frameborder="0", allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture",
      width=560, height=350)

Click [here](https://youtu.be/x8hAYCvVUfE) to watch a demonstration in the youtube.

## Synopsis

To Extract the Grammar:
```
./clifuzzer --get-grammar -o myls-gram.json ./myls
```
To Fuzz:
```
./clifuzzer -f 1000 -g myls-gram.json -o myls.out --invalid-options --invalid-values --log-pass ./myls
```

## Prerequisites
We start by installing a few prerequisite Jupyter extensions.

In [ ]:
%pip install jupyter_contrib_nbextensions jupyter_nbextensions_configurator==0.4.1

In [ ]:
%pip install fuzzingbook

In [ ]:
import sys

In [ ]:
!{sys.executable} -m jupyter contrib nbextension install --sys-prefix

In [ ]:
!{sys.executable} -m jupyter nbextensions_configurator enable

In [ ]:
!{sys.executable} -m jupyter nbextension enable toc2/main

In [ ]:
!{sys.executable} -m jupyter nbextension enable collapsible_headings/main

Please Restart and refresh the Jupyter notebook at this point.

In [1]:
!rm -rf outputdir

In [2]:
!mkdir outputdir

### Compilation

In [3]:
!rm -rf c-lib

In [4]:
!(cd src/c-lib; make)

make: Nothing to be done for `all'.


In [5]:
!cp -r src/c-lib .

## Walk Through

### Step 1. 
We start with the assumption that we have a UNIX command line.

### Step 2.
Consider an example CLI utility `ls` (from [here](https://codereview.stackexchange.com/questions/114479/implementation-of-the-ls-command-with-several-options)) that uses `getopt` to parse its invocations.

In [6]:
!rm myls*

In [7]:
%%writefile myls.c
#include <sys/stat.h>
#include <sys/types.h>
#include <dirent.h>
#include <getopt.h>
#include <grp.h>
#include <pwd.h>
#include <stdbool.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <strings.h>
#include <sysexits.h>
#include <time.h>
#include <unistd.h>
#define program_name "myls"
static char* global_dir = ".";

struct Options
{
    bool using_a;
    bool using_d;
    bool using_h;
    bool using_i;
    bool using_l;
    bool using_p;
    bool using_Q;
    bool using_R;
    bool using_S;
    bool using_t;
    bool using_U;
};

static void init_opts(struct Options* opts)
{
    opts->using_a = false;
    opts->using_d = false;
    opts->using_h = false;
    opts->using_i = false;
    opts->using_l = false;
    opts->using_p = false;
    opts->using_Q = false;
    opts->using_R = false;
    opts->using_S = false;
    opts->using_t = false;
    opts->using_U = false;
}

struct Options get_opts(int count, char* args[])
{
    struct Options opts;
    init_opts(&opts);
    int opt;

    while ((opt = getopt(count, args, "adhilpQRStU")) != -1)
    {
        switch (opt)
        {
            case 'a': opts.using_a = true; break;
            case 'd': opts.using_d = true; break;
            case 'h': opts.using_h = true; break;
            case 'i': opts.using_i = true; break;
            case 'l': opts.using_l = true; break;
            case 'p': opts.using_p = true; break;
            case 'Q': opts.using_Q = true; break;
            case 'R': opts.using_R = true; break;
            case 'S': opts.using_S = true; break;
            case 't': opts.using_t = true; break;
            case 'U': opts.using_U = true; break;
            case '?': exit(EX_USAGE);
        }
    }

    return opts;
}

static void print_permissions(mode_t mode)
{
    putchar((mode & S_IRUSR) ? 'r' : '-');
    putchar((mode & S_IWUSR) ? 'w' : '-');
    putchar((mode & S_IXUSR) ? 'x' : '-');
    putchar((mode & S_IRGRP) ? 'r' : '-');
    putchar((mode & S_IWGRP) ? 'w' : '-');
    putchar((mode & S_IXGRP) ? 'x' : '-');
    putchar((mode & S_IROTH) ? 'r' : '-');
    putchar((mode & S_IWOTH) ? 'w' : '-');
    putchar((mode & S_IXOTH) ? 'x' : '-');
}

static void print_filetype(mode_t mode)
{
    switch (mode & S_IFMT)
    {
        case S_IFREG: putchar('-'); break;
        case S_IFDIR: putchar('d'); break;
        case S_IFLNK: putchar('l'); break;
        case S_IFCHR: putchar('c'); break;
        case S_IFBLK: putchar('b'); break;
        case S_IFSOCK: putchar('s'); break;
        case S_IFIFO: putchar('f'); break;
    }
}

void readable_fs(double size, char* buf)
{
    const char* units[] = { "", "K", "M", "G", "T" };
    int i = 0;

    while (size > 1024)
    {
        size /= 1024;
        ++i;
    }

    sprintf(buf, "%.*f%s", i, size, units[i]);
}

void print_time(time_t mod_time)
{
    // get current time with year
    time_t curr_time;
    time(&curr_time);
    struct tm* t = localtime(&curr_time);
    const int curr_mon = t->tm_mon;
    const int curr_yr = 1970 + t->tm_year;

    // get mod time and year
    t = localtime(&mod_time);
    const int mod_mon = t->tm_mon;
    const int mod_yr = 1970 + t->tm_year;

    // determine format based on years
    const char* format = ((mod_yr == curr_yr)
                       && (mod_mon >= (curr_mon - 6)))
                           ? "%b %e %H:%M"
                           : "%b %e  %Y";

    char time_buf[128];
    strftime(time_buf, sizeof(time_buf), format, t);
    printf("%s", time_buf);
}

struct stat get_stats(const char* filename)
{
    char path[1024];
    sprintf(path, "%s/%s", global_dir, filename);
    struct stat sb;

    if (lstat(path, &sb) < 0)
    {   
        perror(path);
        exit(EX_IOERR);
    }

    return sb;
}

bool is_dir(const char* filename)
{
    struct stat sb = get_stats(filename);

    if (lstat(filename, &sb) < 0)
    {
        perror(filename);
        return false;
    }

    return (sb.st_mode & S_IFDIR) ? true : false;
}

bool is_in_dir(const char* dir, const char* filename)
{
    DIR* dfd = opendir(dir);

    if (!dfd)
    {
        perror(dir);
        return false;
    }

    struct dirent* dp = readdir(dfd);

    while (dp)
    {
        if (strcmp(filename, dp->d_name) == 0)
        {
            closedir(dfd);
            return true;
        }      

        dp = readdir(dfd);
    }

    fprintf(stderr, "file \'%s\' not found\n", filename);

    closedir(dfd);

    return false;
}

void print_name_or_link(const char* filename, struct Options opts, mode_t mode)
{
    if (mode & S_IFLNK)
    {
        char link_buf[512];
        int count = readlink(filename, link_buf, sizeof(link_buf));

        if (count >= 0)
        {
            link_buf[count] = '\0';

            if (opts.using_Q)
            {
                printf(" \"%s\" -> \"%s\"\n", filename, link_buf);
            }
            else
            {
                printf(" %s -> %s \n", filename, link_buf);
            }

            return;
        }
    }

    if (opts.using_Q)
    {
        printf(" \"%s\"", filename);
    }
    else
    {
        printf(" %s", filename);
    }

    if (opts.using_p && is_dir(filename))
    {
        putchar('/');
    }

    putchar('\n');
}

void display_stats(char* dir, char* filename, struct Options opts)
{
    if (!is_in_dir(dir, filename))
    {
        return;
    }

    if (!opts.using_l)
    {
        printf("%s\n", filename);
        return;
    }

    global_dir = dir;

    struct stat sb = get_stats(filename);

    if (opts.using_i)
    {
        printf("%ld ", (long)sb.st_ino);
    }

    print_filetype(sb.st_mode);
    print_permissions(sb.st_mode);
    printf(" %d ", sb.st_nlink);
    printf("%10s ", getpwuid(sb.st_uid)->pw_name);
    printf("%10s", getgrgid(sb.st_gid)->gr_name);
    printf("%10ld ", (long)sb.st_size);

    print_time(sb.st_mtime);
    print_name_or_link(filename, opts, sb.st_mode);
}

bool can_recurse_dir(const char* parent, char* curr)
{
    if (!strcmp(".", curr) || !strcmp("..", curr))
    {
        return false;
    }

    char path[2048];
    sprintf(path, "%s/%s", parent, curr);
    struct stat sb;

    if (lstat(path, &sb) < 0)
    {
        perror(path);
        exit(EX_IOERR);
    }

    return S_ISDIR(sb.st_mode);
}

void recurse_dirs(char* dir, struct Options opts)
{
    DIR* dfd = opendir(dir);
    struct dirent* dp = readdir(dfd);

    printf("\n%s:\n", dir);

    while ((dp = readdir(dfd)))
    {
        const bool omit_hidden = !opts.using_a && dp->d_name[0] == '.';

        if (!omit_hidden)
        {
            if (opts.using_l)
            {
                display_stats(dir, dp->d_name, opts);
            }
            else
            {
                printf("%s\n", dp->d_name);
            }
        }

        if (can_recurse_dir(dir, dp->d_name))
        {
            char next[1024];
            sprintf(next, "%s/%s", dir, dp->d_name);
            recurse_dirs(next, opts);
        }
    }

    closedir(dfd);
}

static int cmp_lex(const void* p1, const void* p2)
{
    const char* str1 = *(const void**)p1;
    const char* str2 = *(const void**)p2;

    return strcasecmp(str1, str2);
}

static int cmp_time(const void* p1, const void* p2)
{
    const char* str1 = *(const char**)p1;
    const char* str2 = *(const char**)p2;

    time_t time1 = get_stats(str1).st_mtime;
    time_t time2 = get_stats(str2).st_mtime;

    return time1 < time2;
}

static int cmp_size(const void* p1, const void* p2)
{
    const char* str1 = *(const char**)p1;
    const char* str2 = *(const char**)p2;

    long int size1 = get_stats(str1).st_size;
    long int size2 = get_stats(str2).st_size;

    return size1 < size2;
}

void display_dir(char* dir, struct Options opts)
{
    DIR* dfd = opendir(dir);
    struct dirent* dp = readdir(dfd);
    long curr_alloc_amt = 30000;
    char** dir_arr = malloc(curr_alloc_amt * sizeof(char*));

    if (!dir_arr)
    {
        abort();
    }

    long int count = 0;

    while (dp)
    {
        const bool omit_hidden = !opts.using_a && dp->d_name[0] == '.';

        if (!omit_hidden)
        {
            if (count >= curr_alloc_amt)
            {
                curr_alloc_amt *= 2;
                dir_arr = realloc(dir_arr, curr_alloc_amt * sizeof(char*));

                if (!dir_arr)
                {
                    abort();
                }
            }

            dir_arr[count] = dp->d_name;
            count++;
        }

        dp = readdir(dfd);
    }

    global_dir = dir;

    if (!opts.using_U && opts.using_t)
    {
        qsort(dir_arr, count, sizeof(char*), cmp_time);
    }
    else if (!opts.using_U && opts.using_S)
    {
        qsort(dir_arr, count, sizeof(char*), cmp_size);
    }
    else if (!opts.using_U)
    {
        qsort(dir_arr, count, sizeof(char*), cmp_lex);
    }

    for (long int i = 0; i < count; ++i)
    {
        display_stats(dir, dir_arr[i], opts);
    }

    closedir(dfd);
    free(dir_arr);
}

void scan_dirs(int count, char* args[], struct Options opts)
{
    if (opts.using_d)
    {
        const bool no_dirs_given = (count - optind) == 0;

        if (no_dirs_given)
        {
            display_stats(".", ".", opts);
        }

        // loop through directories
        for (int i = optind; i < count; ++i)
        {
            display_stats(".", args[i], opts);
        }

        return;
    }

    // no other arguments
    if (!opts.using_R && (optind == count))
    {
        display_dir(".", opts);
    }

    if (opts.using_R && !opts.using_d)
    {
        recurse_dirs(".", opts);
        return;
    }

    const bool multiple_dirs = (count - optind) >= 2;

    // loop through directories
    for (int i = optind; i < count; ++i)
    {
        if (!is_dir(args[i]))
        {
            display_stats(".", args[i], opts);
            continue;
        }

        // display directory name
        //   for multiple directories
        if (multiple_dirs)
        {
            printf("\n%s:\n", args[i]);
        }

        if (!is_in_dir(".", args[i]))
        {
            continue;
        }

        display_dir(args[i], opts);
    }
}

void
usage (int status)
{
  if (status != EXIT_SUCCESS)
    fprintf (stderr, ("Try `%s -h' for more information.\n"), program_name);
  else {
      printf (("Usage: %s [OPTION]... [FILE]...\n"), program_name);
      fputs (("\
List information about the FILEs (the current directory by default).\n\
Sort entries alphabetically if none of -cftuvSUX nor --sort.\n\
\n\
"), stdout);
      fputs (("\
Mandatory arguments to long options are mandatory for short options too.\n\
"), stdout);
      fputs (("\
  -a                         do not ignore entries starting with .\n\
"), stdout);
      fputs (("\
  -d,                        list directory entries instead of contents,\n\
                               and do not dereference symbolic links\n\
"), stdout);
      fputs (("\
  -i                         print the index number of each file\n\
"), stdout);
      fputs (("\
  -l                         use a long listing format\n\
"), stdout);
      fputs (("\
  -p                         append / indicator to directories\n\
"), stdout);
      fputs (("\
  -Q                         enclose entry names in double quotes\n\
"), stdout);
      fputs (("\
  -R                         list subdirectories recursively\n\
"), stdout);
      fputs (("\
  -S                         sort by file size\n\
"), stdout);
      fputs (("\
  -t                         sort by modification time\n\
"), stdout);
      fputs (("\
  -U                         do not sort; list entries in directory order\n\
"), stdout);
    }
  exit (status);
}


int main(int argc, char* argv[])
{ 
    struct Options opts = get_opts(argc, argv);
    if (opts.using_h) {
        usage(EXIT_SUCCESS);
    }
    scan_dirs(argc, argv, opts);

    return 0;
}

Writing myls.c


We now compile the `myls.c` source file, and enable coverage.

In [8]:
!cc --coverage myls.c -o myls

### Step 3.


`myls` takes a set of command line options. As is the usual case with well behaved UNIX utilities, passing the command line parameter `-h` to `myls` will list its accepted options.

In [9]:
!./myls -h

Usage: myls [OPTION]... [FILE]...
List information about the FILEs (the current directory by default).
Sort entries alphabetically if none of -cftuvSUX nor --sort.

Mandatory arguments to long options are mandatory for short options too.
  -a                         do not ignore entries starting with .
  -d,                        list directory entries instead of contents,
                               and do not dereference symbolic links
  -i                         print the index number of each file
  -l                         use a long listing format
  -p                         append / indicator to directories
  -Q                         enclose entry names in double quotes
  -R                         list subdirectories recursively
  -S                         sort by file size
  -t                         sort by modification time
  -U                         do not sort; list entries in directory order


We observe that `myls` can have multiple options in its invocation e.g., `-U`, `-i` etc.

### Step 4.
Let us create a random fuzzer for fuzzing the `ls` command line. Given an input space such as the command line, the traditional fuzzers (e.g. Miller et al. 1990) simply produce a random string, which is then fed into the command. Any crashes observed are recorded. We first create a random input generator as below.

In [10]:
import string, random
def fuzzer(max_length, choices):
    string_length = random.randrange(1, max_length + 1)
    return ''.join([random.choice(choices)
                    for c in range(string_length)])

In [11]:
fuzzer(100, string.printable)

",%2m\t!8Z/z\x0c\nU\n9xnzR!#oI%>A+KfV=guK'k+9AySI`PE.}f52XrX0^^)TR}3|"

We can now use this fuzzer to produce a command line fuzzer. We want to produce command lines that are composed of options option arguments and stand alone arguments. Given our knowledge of command lines, we know that the command line options start with `-` and are composed of ASCII letters.

In [12]:
chars = [c for c in (string.ascii_letters + "-_ ")]
invocations = []
for i in range(10):
    s = fuzzer(100, chars)
    invocations.append('%s %s' % ('./myls', s))

We then use the generated invocations.

In [13]:
import os, sys

In [14]:
for i in invocations:
    os.system(i)

./ZzLEFTaqInSeqRtXBYCdDSWMdaAvzvWBJ_feImnVZoWo: No such file or directory
./Pqj: No such file or directory
./yPkOvbRENYSYt_oiQPyJenZNMmLxCEVPKgmGp: No such file or directory
./fS-AigYesQG_I--LdSPuzCVazWDXFCyHxyJcimsVadHvBQaC_: No such file or directory
./YDjucql_YpaNdOvBdnaGkwjD: No such file or directory
./uaCKImwWJsMbfHl: No such file or directory
./pt: No such file or directory
./QpYIXOp-SnHQtpptbEECD_GrmvqVzrRZ-IEShFYiYQCPg_BC-lmLCjZkdRJxQJ: No such file or directory
./kahjdCtbHPppDOMREY-HtensPBfgV: No such file or directory
./QV_Jrj: No such file or directory


Most (almost all) invocations result in an error condition. This is expected, however.

### Step 5.

We now look at the coverage obtained. Given that we are fuzzing `myls` randomly, the coverage obtained is limited, because a number of possible options and option arguments may not be present in the fuzz invocations.

In [15]:
!gcov ./myls

File 'myls.c'
Lines executed:24.21% of 285
Creating 'myls.c.gcov'



In the `.gcov` file, each line is prefixed with the number of times it was called (`-` stands for a non-executable line, `#####` stands for zero) as well as the line number.

In [16]:
lines = open('myls.c.gcov').readlines()
for i in range(len(lines)):
    print(lines[i], end='')

        -:    0:Source:myls.c
        -:    0:Graph:./myls.gcno
        -:    0:Data:./myls.gcda
        -:    0:Runs:11
        -:    0:Programs:1
        -:    1:#include <sys/stat.h>
        -:    2:#include <sys/types.h>
        -:    3:#include <dirent.h>
        -:    4:#include <getopt.h>
        -:    5:#include <grp.h>
        -:    6:#include <pwd.h>
        -:    7:#include <stdbool.h>
        -:    8:#include <stdio.h>
        -:    9:#include <stdlib.h>
        -:   10:#include <string.h>
        -:   11:#include <strings.h>
        -:   12:#include <sysexits.h>
        -:   13:#include <time.h>
        -:   14:#include <unistd.h>
        -:   15:#define program_name "myls"
        -:   16:static char* global_dir = ".";
        -:   17:
        -:   18:struct Options
        -:   19:{
        -:   20:    bool using_a;
        -:   21:    bool using_d;
        -:   22:    bool using_h;
        -:   23:    bool using_i;
        -:   24:    bool using_l;
        -:   25:    b

### Step 6.

The table below lists the difference in code coverage between fuzzing `coreutils` without options using `AFL++` and fuzzing `coreutils` with options using `clifuzzer`. We see that `clifuzzer` is able to achieve better Line and Branch Coverage.

| Utility |#lines |#branches| AFL++ Line Coverage | AFL++ Branch Coverage | CLIFuzz Line Coverage | CLIFuzz Branch Coverage |
|---|---|---|---|---|---|---|
| as (v2.37) | 73643 | 53008 | 17.41 | 14.15 | 23.00 | 19.24|
| bc (v1.07.1) | 3529 | 1950 | 57.04 | 66.20 | 56.59 | 63.57|
| bison (v3.8) | 18938 | 13768 | 13.10 | 12.07 | 37.35 | 34.87|
| cat (v9.0) | 1044 | 784 | 33.94 | 22.71 | 35.81 | 25.36|
| cmp (v3.8) | 1604 | 1160 | 16.18 | 13.57 | 17.07 | 15.26|
| col (v2.37.2) | 949 | 790 | 32.31 | 33.67 | 46.75 | 48.79|
| colcrt (v2.37.2) | 168 | 97 | 65.34 | 77.85 | 65.00 | 81.10|
| column(v2.37.2) | 1638 | 1274 | 26.06 | 27.69 | 40.61 | 42.11|
| colrm (v2.37.2) | 659 | 615 | 25.36 | 28.61 | 33.46 | 39.16|
| comm (v9.0) | 1044 | 758 | 28.94 | 29.23 | 34.40 | 34.70|
| cut (v9.0) | 1240 | 1028 | 13.83 | 11.80 | 45.03 | 37.97|
| dc (v1.4.1) | 1920 | 1131 | 81.97 | 85.21 | 84.96 | 92.19|
| diff (v3.8) | 9667 | 7382 | 15.34 | 9.45 | 36.41 | 31.58|
| expand (v9.0) | 1297 | 930 | 28.08 | 29.08 | 30.07 | 30.13|
| fmt (v9.0) | 1189 | 1007 | 23.63 | 21.64 | 37.39 | 35.63|
| fold (v9.0) | 953 | 848 | 23.65 | 21.17 | 37.61 | 35.55|
| gdb (v11.1) | 81215 | 58852 | 11.97 | 8.98 | 11.71 | 8.81 |
| grep (v3.7) | 10331 | 7790 | 24.37 | 19.97 | 49.09 | 44.01|
| head (v9.0) | 1291 | 980 | 19.60 | 13.20 | 33.09 | 20.81|
| join (v9.0) | 1490 | 1158 | 36.26 | 30.37 | 49.62 | 43.51|
| look (v2.37.2) | 151 | 107 | 62.72 | 77.06 | 73.31 | 87.65|
| m4 (v1.4.19) | 12521 | 9304 | 17.46 | 14.42 | 23.22 | 18.99|
| nl (v9.0) | 4881 | 3876 | 20.63 | 19.45 | 46.86 | 40.23|
| nm (v2.37) | 53936 | 38521 | 10.93 | 9.18 | 10.51 | 9.24|
| od (v9.0) | 2337 | 1866 | 27.98 | 19.74 | 52.29 | 43.57|
| paste (v9.0) | 954 | 767 | 32.45 | 22.61 | 33.90 | 25.06|
| pr (v9.0) | 2589 | 2181 | 27.08 | 18.61 | 46.36 | 36.44|
| ptx (v9.0) | 6013 | 4872 | 30.34 | 26.20 | 38.52 | 35.53|
| rev (v2.37.2) | 103 | 57 | 71.62 | 86.97 | 67.83 | 86.97|
| sdiff (v3.8) | 1761 | 1084 | 12.04 | 9.39 | 13.34 | 11.12|
| sort (v9.0) | 4113 | 2840 | 21.19 | 16.76 | 44.62 | 41.19|
| spell (v1.1) | 306 | 185 | 53.59 | 56.22 | 80.39 | 81.62|
| strings (v2.37) | 53349 | 38055 | 1.14 | 0.55 | 8.03 | 6.77 |
| strip (v2.37) | 60632 | 42785 | 9.61 | 7.76 | 16.01 | 13.57|
| tac (v9.0) | 4928 | 3825 | 16.49 | 11.31 | 31.57 | 20.52|
| tail (v9.0) | 2467 | 1855 | 13.35 | 13.38 | 22.59 | 19.13|
| tee (v9.0) | 963 | 735 | 32.18 | 18.25 | 34.34 | 21.13|
| tr (v9.0) | 1503 | 1215 | 43.25 | 27.44 | 45.70 | 31.84|
| troff (v1.22.4) | 16731 | 15024 | 48.75 | 40.78 | 59.71 | 51.88|
| tsort (v9.0) | 1078 | 813 | 29.70 | 28.79 | 29.70 | 29.48|
| unexpand(v9.0) | 1000 | 795 | 31.64 | 22.88 | 37.70 | 30.10|
| uniq (v9.0) | 1160 | 974 | 30.67 | 23.55 | 37.43 | 31.03|
| wc (v9.0) | 1446 | 1041 | 36.64 | 30.15 | 40.58 | 34.76|
| xargs (v4.8.0) | 2128 | 1501 | 30.68 | 28.27 | 51.73 | 46.10|

### Step 7.
How CLIFuzz works (slides)

CLIFuzz extracts the options and arguments details from utilities, generates grammars from the extracted information and uses the grammars to generate invocation strings for purpose of fuzzing the utilities. Once the grammar has been saved, it can reuse that grammar to fuzz as many times as wished.

The slides below describe the steps in more details

![CLIFuzz's steps](demo_images/wf1.png)

The options and arguments extraction and can be further divided into four steps -

![Grammar construction steps](demo_images/gs4.png)

### Step 8.
To invoke the CLIFuzzer, the command is `clifuzzer`. It extracts the options from the utility given as its argument, and generates the invocation grammar for that utility. It can also run the fuzzer on the utility using its invocation grammar. The `--help` output describes its usage.

In [17]:
 !{sys.executable} ./clifuzzer --help

usage: clifuzzer [-h]
                 [--get-grammar | --get-options | -f FUZZ | --get-coverage | --fuzz-coverage FUZZ_COVERAGE | --get-manual-coverage]
                 [-o O] [-g GRAM_FILE] [--log-pass] [--invalid-options]
                 [--invalid-values] [--seed SEED]
                 binary

Run Configuration fuzzing on the given binary

positional arguments:
  binary                Binary or path to Binary to fuzz on

optional arguments:
  -h, --help            show this help message and exit
  --get-grammar         Print the grammar extracted from options
  --get-options         Print a list of cmdline options available in the
                        binary
  -f FUZZ, --fuzz FUZZ  No. of times the fuzzer should fuzz the binary and
                        note unexpected behaviour including crashes and
                        interesting return codes
  --get-coverage        Extract the coverage achieved in the tool
  --fuzz-coverage FUZZ_COVERAGE
                        Run fu

### Step 9
Here is a demonstation of invoking `clifuzzer` on `myls`, extracting its options and producing the invocation grammar.


In [18]:
!{sys.executable} ./clifuzzer --get-grammar -o myls-gram.json ./myls

Test binary: ./myls!
Writing to: myls-gram.json
Writing done!


The grammar extracted is given below.

In [19]:
!cat myls-gram.json

{
    "<start>": [
        "<option><arguments>"
    ],
    "<option>": [
        "(<other_option>)*"
    ],
    "<arguments>": [
        ""
    ],
    "<other_option>": [
        " -a",
        " -d",
        " -h",
        " -i",
        " -l",
        " -p",
        " -Q",
        " -R",
        " -S",
        " -t",
        " -U"
    ]
}

### Step 10.

We show how to produce inputs from this grammar and show the resulting command-line invocations

In [45]:
import fuzzingbook.GrammarFuzzer as grammarfuzzer
import fuzzingbook.Grammars as grammars
import json

In [54]:
with open('myls-gram.json') as f:
    g = grammars.convert_ebnf_grammar(json.load(fp=f))
for k in g:
    print(k, g[k])

<start> ['<option><arguments>']
<option> ['<symbol-1>']
<arguments> ['']
<other_option> [' -a', ' -d', ' -h', ' -i', ' -l', ' -p', ' -Q', ' -R', ' -S', ' -t', ' -U']
<symbol> ['<other_option>']
<symbol-1> ['', '<symbol><symbol-1>']


In [53]:
gf = grammarfuzzer.GrammarFuzzer(g)
for i in range(10):
    print(repr(gf.fuzz()))

''
' -S -l -p'
' -d -d'
' -S -a -Q -d'
' -a -t'
' -Q -p'
''
' -U'
' -l'
''


### Step 11. 
At this point, we can use the grammar we mined to fuzz `myls` with CLIFuzz. We start by clearing out the previous fuzzing output if any, and recreating the file strucutre.

In [20]:
!rm -rf FILE_backup/

In [21]:
!mkdir -p FILE_backup/emptydir
for f in ['HelloWorld.py', 'README', 'emptyfile', 'bible.txt', 'world192.txt', 'testopt', 'audio.wav',   
'largeaudio.wav', 'image.jpg', 'E.coli', 'as.s', 'bison.y', 'dc.txt', 'gdb.txt', 'bc.txt',      
's0', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',  
's15', 's16', 's17', 's18', 's19', 's20', 's21', 's22', 's23', 's24', 's25', 's26', 's27',      
's28', 's29', 'l0', 'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'l7', 'l8', 'l9', 'l10', 'l11', 'l12',  
'l13', 'l14', 'l15', 'l16', 'l17', 'l18', 'l19', 'l20', 'l21', 'l22', 'l23', 'l24', 'l25',      
'l26', 'l27', 'l28', 'l29']:
    !touch FILE_backup/{f}

Next, we invoke the `clifuzzer` with the previously extracted grammar as the input.

In [22]:
!{sys.executable} ./clifuzzer \
   -f 1000 -g myls-gram.json \
   -o myls.out \
   --invalid-options \
   --invalid-values \
   --log-pass ./myls

Test binary: ./myls!
Writing to: myls.out
Reading grammar from: myls-gram.json
Fuzzing ./myls 1000 times now starting at 2022-06-28 02:44:55.898892!
attempt  0 ./myls -S
attempt  1 ./myls -a -l -R -h -U -Q -t -d -p -i -Q
attempt  2 ./myls
attempt  3 ./myls -a -p
attempt  4 ./myls
attempt  5 ./myls -p -a
attempt  6 ./myls -h
attempt  7 ./myls
attempt  8 ./myls -h
attempt  9 ./myls
attempt  10 ./myls -S
attempt  11 ./myls -p
attempt  12 ./myls
attempt  13 ./myls
attempt  14 ./myls -l
attempt  15 ./myls -t
attempt  16 ./myls -U
attempt  17 ./myls -l
attempt  18 ./myls
attempt  19 ./myls
attempt  20 ./myls -Q -U -p
attempt  21 ./myls
attempt  22 ./myls
attempt  23 ./myls -R
attempt  24 ./myls
attempt  25 ./myls
attempt  26 ./myls
attempt  27 ./myls -i -i
attempt  28 ./myls
attempt  29 ./myls
attempt  30 ./myls
attempt  31 ./myls -l -t
attempt  32 ./myls
attempt  33 ./myls -U -d
attempt  34 ./myls
attempt  35 ./myls
attempt  36 ./myls
attempt  37 ./myls -a -S -h
attempt  38 ./myls
attempt  

attempt  359 ./myls -S -h
attempt  360 ./myls
attempt  361 ./myls
attempt  362 ./myls -i
attempt  363 ./myls -i -i -R
attempt  364 ./myls -S
attempt  365 ./myls -i
attempt  366 ./myls
attempt  367 ./myls
attempt  368 ./myls -t -l -d -Q -a
attempt  369 ./myls
attempt  370 ./myls -Q
attempt  371 ./myls
attempt  372 ./myls -Q -l
attempt  373 ./myls
attempt  374 ./myls
attempt  375 ./myls -i -t
attempt  376 ./myls -d -i
attempt  377 ./myls -h
attempt  378 ./myls
attempt  379 ./myls -i
attempt  380 ./myls -l
attempt  381 ./myls
attempt  382 ./myls
attempt  383 ./myls -h
attempt  384 ./myls
attempt  385 ./myls
attempt  386 ./myls
attempt  387 ./myls
attempt  388 ./myls -i -Q -l
attempt  389 ./myls -Q -a
attempt  390 ./myls -a
attempt  391 ./myls
attempt  392 ./myls
attempt  393 ./myls -l -R -U -R -Q
attempt  394 ./myls -R
attempt  395 ./myls -Q
attempt  396 ./myls
attempt  397 ./myls
attempt  398 ./myls
attempt  399 ./myls -t
attempt  400 ./myls
attempt  401 ./myls -R
attempt  402 ./myls
att

attempt  721 ./myls
attempt  722 ./myls
attempt  723 ./myls
attempt  724 ./myls
attempt  725 ./myls -U -h
attempt  726 ./myls
attempt  727 ./myls -S -a
attempt  728 ./myls
attempt  729 ./myls -h -i
attempt  730 ./myls
attempt  731 ./myls
attempt  732 ./myls
attempt  733 ./myls -S -R
attempt  734 ./myls -Q -l -l -l -R -p -t
attempt  735 ./myls
attempt  736 ./myls -Q -h -d
attempt  737 ./myls -S
attempt  738 ./myls
attempt  739 ./myls
attempt  740 ./myls
attempt  741 ./myls
attempt  742 ./myls
attempt  743 ./myls
attempt  744 ./myls -a -U -S -U
attempt  745 ./myls
attempt  746 ./myls -d
attempt  747 ./myls -i -a
attempt  748 ./myls -i
attempt  749 ./myls
attempt  750 ./myls
attempt  751 ./myls -t
attempt  752 ./myls
attempt  753 ./myls
attempt  754 ./myls
attempt  755 ./myls
attempt  756 ./myls
attempt  757 ./myls
attempt  758 ./myls -p
attempt  759 ./myls -R
attempt  760 ./myls -l
attempt  761 ./myls
attempt  762 ./myls -h
attempt  763 ./myls -d -p -h
attempt  764 ./myls
attempt  765 ./

### Step 12.
Let us check whether our fuzzing run has improved the coverage from pure random fuzzing.

In [23]:
!gcov ./myls

File 'myls.c'
Lines executed:83.86% of 285
Creating 'myls.c.gcov'



### Step13.
Video ScreenGrab of the execution of `clifuzzer` on `myls`

A video screengrab of `clifuzzer`'s execution on `myls` is available at https://github.com/clifuzzer/fse2022-clifuzzer/blob/main/demo_images/myls_screenrecording.mp4

### Step14

We have demonstrated how `clifuzzer` works, and can be used to improve the fuzz coverage of command line utilities.